In [10]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from stop_words import get_stop_words
from nltk.corpus import stopwords
import pickle
from utils import *
import pandas as pd
import numpy as np
import gc
from datetime import datetime

[nltk_data] Downloading package wordnet to /home/sdey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sdey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
class leos_member_basic(object):
    def __init__(self, region, ceo, state, name, title, office, got, party, twitter_handle, personalhandle, leo):
        self.region = region
        self.ceo = ceo
        self.name = name
        self.title = title
        self.office = office
        self.got = got
        self.party = party
        self.twitter_handle = twitter_handle
        self.personalhandle = personalhandle
        self.author_id = None
        self.leo = leo
        self.twitter_id_set = set()
        
class leos_member_twitter(leos_member_basic):
    def __init__(self, region, ceo, state, name, title, office, party, got, twitter_handle, personalhandle, leo):
        super(leos_member_twitter, self).__init__(region, ceo, state, name, title, office, got, party, twitter_handle, personalhandle, leo)
        self.raw_twitter_record = []
    
    def get_tweets(self,handle):
        # print('start collecting tweets for handle: ' + handle)
        max_id = None
        for _ in range(16):
            try:
                t = api.GetUserTimeline(screen_name=handle, count=200, max_id=max_id, include_rts=True)
                tweets = [i.AsDict() for i in t]
                for t in tweets:
                    try:
                        if t["id"] not in self.twitter_id_set:
                            max_id = t["id"]
                            self.raw_twitter_record.append(t)
                            self.twitter_id_set.add(t["id"])
                    except:
                        pass
            except:
                pass

In [12]:
## get all stop words

en_stop =[]
en_stop1 = get_stop_words("english")
en_stop2 = stopwords.words('english')
en_stop = [en_stop1,en_stop2,["youd","doesnt","youll","will","wont","aint","ith","their","they're","is",
                              "was","been","not","they","it","have","will","has","by","for","start","one",
                              "thing","learned","im","youve","theyre","their","things" ,"things","can't",
                              "can","cant","will","go","going","let","lets","let's","say","says","know",
                              "talk","talked","talks","dont","think","said","something","something","wont",
                              "people","make","want","went","goes","people","person","like","come","from",
                              "need","us","didnt","woudldnt","today","discuss","new","great","support","see",
                              "did","must","ha","wa","im","u","new","continue","tomorrow","join","everyone",
                              "time","great","may","see","area","good","across","take","tell","amazing",
                              "below","behind","next","hour","thank","thanks","come"]]
en_stop = [item for sublist in en_stop for item in sublist]

print(en_stop)
wordnet_lemmatizer = WordNetLemmatizer()


def get_time(single_tweet):
    month_dict ={'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    time_string = single_tweet['created_at']
    all_time_items = time_string.split()
    return month_dict[all_time_items[1]], all_time_items[2], all_time_items[-1]

def preprocess(text):
    clean_text = ""
    for eachone in text:
        try:
            eachone.encode('ascii')
            clean_text += eachone
        except:
            pass

    new_text = []
    for each_word in clean_text.split():
        if each_word.isalpha():
            lower_word = wordnet_lemmatizer.lemmatize(each_word.lower())
            if lower_word not in en_stop:
                new_text.append(lower_word)
    return ' '.join(new_text)


# disregarding retweets
clean_tweet_dict   = dict()


['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

In [13]:

# load the dictionaries of tweets and combine them

total_tweet_dict = pickle.load(open('layer_tweet_dict', 'rb'))


0

In [15]:

clean_raw_tweet_dict = dict()

## Create the empty lists that willl hold our variables that we will bind into a dataframe
region = []
ceo = []
name = []
title = []
office = []
got = []
party = []
twitter_handle = []
personal_handle = []
IDlist = []
author_id = []
leo = []
monthlist = []
yearlist = []
daylist = []
totalArticle = []

count = 0
for key, layer_member in total_tweet_dict.items():
    tweet_title = key
    raw_tweet_list = layer_member.raw_twitter_record
    
    # call each tweet for each layer member
    for each_tweet in raw_tweet_list:
        # print(each_tweet)
        
        # keep only NON-retweets and NON-quote tweets
        if not ('retweeted_status' in each_tweet or 'quoted_status' in each_tweet):
            each_raw_tweet = each_tweet['full_text']
            month, day, year = get_time(each_tweet)
            
            #clean each tweet using preprocess function from above
            tmp_post_clean_tweet = preprocess(each_raw_tweet)
            
            if len(tmp_post_clean_tweet) > 0:
                clean_raw_tweet_dict[tmp_post_clean_tweet] = each_raw_tweet
                
                region.append(layer_member.region)
                ceo.append(layer_member.ceo)
                name.append(layer_member.name)
                title.append(layer_member.title)
                office.append(layer_member.office)
                got.append(layer_member.got)
                party.append(layer_member.party)
                twitter_handle.append(layer_member.twitter_handle)
                personal_handle.append(layer_member.personalhandle)
                IDlist.append(each_tweet['id'])
                leo.append(layer_member.leo)
                monthlist.append(month)
                yearlist.append(year)
                daylist.append(day)
                totalArticle.append(tmp_post_clean_tweet)

                try:
                    author_id.append(each_tweet['author_id'])
                except:
                    author_id.append(each_tweet['user']['id_str'])


In [16]:
gc.collect()

22

In [17]:
print(len(totalArticle))

116158


In [18]:
# create dataset 

# df = pd.DataFrame(list(zip(totalArticle,IDlist,handle ,name,fname,lname,party,state,house,daylist,monthlist,yearlist)), 
#                columns =['tweet',"doc_id", "handle",'name',"firstname","lastname","party","state","chamber","day","month","year"])
# print(df.head())

df = pd.DataFrame(list(zip(totalArticle,IDlist,twitter_handle,author_id,region,ceo,name,title,office,got,party,personal_handle,leo,monthlist,yearlist,daylist)), 
               columns=['tweet','doc_id','handle','author_id','region','ceo','name','title','office','got','party','personal_handle','leo','month','year','day'])
print(df.head())


                                               tweet               doc_id  \
0  secretary state office provide unofficial elec...  1539249005917614083   
1  primary runoff election important voter voted ...  1537812869395406849   
2  ahead june primary runoff secretary merrill re...  1537465239813468160   
3  ahead june primary runoff secretary merrill re...  1537129963584901127   
4  secretary merrill remind alabama voter last da...  1536364016464961536   

         handle author_id region ceo         name               title  \
0  johnhmerrill  40788002  south      john merill  secretary of state   
1  johnhmerrill  40788002  south      john merill  secretary of state   
2  johnhmerrill  40788002  south      john merill  secretary of state   
3  johnhmerrill  40788002  south      john merill  secretary of state   
4  johnhmerrill  40788002  south      john merill  secretary of state   

                                     office            got party  \
0  office of sectary of state 

In [9]:
df.to_csv('clean_dataset.csv')